In [ ]:
import tensorflow as tf
import numpy as np
#from matplotlib.colors import ListedColormap #Uncomment these if you have installed matplotlib (pip3 install matplotlib)
#%pylab inline

In [ ]:
with open("x_circles.data",'rb') as f:
    x = np.load(f)
with open("y_circles.data",'rb') as f:
    y = np.load(f).reshape(-1,1)

In [ ]:
x.shape, y.shape

In [ ]:
#scatter(x[:,0], x[:,1], c=y, cmap= ListedColormap(['#FF0000', '#0000FF']))

In [ ]:
class LogisticRegression():
    def __init__(self, batchsize, xsize, optimizer=tf.train.GradientDescentOptimizer):
        self.learning_rate = tf.Variable(0.0, dtype=tf.float32, trainable=False)
        self.x = tf.placeholder(shape=(batchsize, xsize), dtype=tf.float32, name="x")
        self.y = tf.placeholder(shape=(batchsize,1), dtype=tf.float32, name="y")
        w = tf.get_variable("w", shape=(xsize,1))
        b = tf.get_variable("b", shape=(1))
        logit = tf.add(tf.matmul(self.x,w),b) #Broadcast
        self.proba = tf.sigmoid(logit)
        self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=self.y)) 
        self.train = optimizer(self.learning_rate).minimize(self.loss)

In [ ]:
initializer = tf.random_uniform_initializer(minval=-0.05, maxval=0.05)
with tf.variable_scope("LR", reuse=None, initializer=initializer):
    lr = LogisticRegression(100,2)

In [ ]:
[(t.name, t.get_shape().as_list()) for t in tf.trainable_variables()]

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer()) #Set w,b to random values from distribution
sess.run(tf.assign(lr.learning_rate, 0.1))

In [ ]:
output = sess.run([lr.loss, lr.train], feed_dict={lr.x:x, lr.y:y})
print(output)

In [ ]:
for i in range(10000):
    loss, proba, _ = sess.run([lr.loss, lr.proba, lr.train], feed_dict={lr.x:x, lr.y:y})
    if i % 1000 == 0:
        proba = sess.run(lr.proba, feed_dict={lr.x:x, lr.y:y})
        accuracy = (np.where(proba >= 0.5, 1, 0) == y).mean()
        print(loss, accuracy)

In [ ]:
sess.run(tf.global_variables_initializer()) #Make weights random again
sess.run(tf.assign(lr.learning_rate, 1.0))

In [ ]:
for i in range(10000):
    loss, proba, _ = sess.run([lr.loss, lr.proba, lr.train], feed_dict={lr.x:x, lr.y:y})
    accuracy = (np.where(proba >= 0.5, 1, 0) == y).mean()
    if i % 1000 == 0:
        print(loss, accuracy)

In [ ]:
class MLP():
    def __init__(self, batchsize, xsize, optimizer=tf.train.GradientDescentOptimizer):
        self.learning_rate = tf.Variable(0.0, dtype=tf.float32, trainable=False)
        self.x = tf.placeholder(shape=(batchsize, xsize), dtype=tf.float32, name="x")
        self.y = tf.placeholder(shape=(batchsize,1), dtype=tf.float32, name="y")
        
        #Hidden Layer
        w = tf.get_variable("w", shape=(xsize, 2))
        b = tf.get_variable("b", shape=(2))
        hidden = tf.sigmoid(tf.matmul(self.x,w) + b) 
        
        w2 = tf.get_variable("w2", shape=(2,1))
        b2 = tf.get_variable("b2", shape=(1))
        logit = tf.matmul(hidden,w2) + b2
        
        self.proba = tf.sigmoid(logit)
        self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=self.y)) 
        self.train = optimizer(self.learning_rate).minimize(self.loss)

In [ ]:
with tf.variable_scope("MLP", reuse=None, initializer=initializer):
    mlp = MLP(100, 2)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.assign(mlp.learning_rate, 0.1))

In [ ]:
for i in range(10000):
    loss, proba,  _  = sess.run([mlp.loss, mlp.proba, mlp.train], feed_dict={mlp.x:x, mlp.y:y})
    if i % 1000 == 0:
        accuracy = (np.where(proba >= 0.5, 1, 0) == y).mean()                                                       
        print(loss, accuracy)

In [ ]:
sess.run(tf.global_variables_initializer()) #Make weights random again
sess.run(tf.assign(mlp.learning_rate, 1.0))

In [ ]:
for i in range(20000):
    loss, proba, _  = sess.run([mlp.loss, mlp.proba, mlp.train], feed_dict={mlp.x:x, mlp.y:y})
    if i % 1000 == 0:
        accuracy = (np.where(proba >= 0.5, 1, 0) == y).mean()                                                       
        print(loss, accuracy)

In [ ]:
with tf.variable_scope("MLP_adam", reuse=None, initializer=initializer):
    mlp = MLP(100, 2, optimizer=tf.train.AdamOptimizer)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.assign(mlp.learning_rate,0.001))

In [ ]:
for i in range(10000):
    loss, proba,  _  = sess.run([mlp.loss, mlp.proba, mlp.train], feed_dict={mlp.x:x, mlp.y:y}) 
    if i % 1000 == 0:
        accuracy = (np.where(proba >= 0.5, 1, 0) == y).mean()                                                       
        print(loss, accuracy)

In [ ]:
sess.run(tf.global_variables_initializer()) #Make weights random again
sess.run(tf.assign(mlp.learning_rate, 0.01))

In [ ]:
for i in range(10000):
    loss, proba,  _  = sess.run([mlp.loss, mlp.proba, mlp.train], feed_dict={mlp.x:x, mlp.y:y}) 
    if i % 1000 == 0:
        accuracy = (np.where(proba >= 0.5, 1, 0) == y).mean()                                                       
        print(loss, accuracy)

In [ ]:
# Baseline optimizer: GradientDescent with starting learning rate of 0.1
# Often the easiest optimizer out of the box: AdamOptimizer with starting learning rate of 0.001
# To do quick checks of learning rate starting values try going up or down 10x (so check within 1.0, 0.1, 0.01)
# Some have found Adam to learn faster, but SGD to eventually get better accuracy
# Google state of the art Neural Machine Translation uses Adam for most of the training
# and then finishes with some SGD (and then Reinforcement Learning!!)
# https://arxiv.org/pdf/1609.08144v2.pdf

In [ ]:
x = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,1,1,0]).reshape(-1,1)

In [ ]:
#scatter(x[:,0], x[:,1], c=y, cmap= ListedColormap(['#FF0000', '#0000FF']))

In [ ]:
class MLP3():
    def __init__(self, batchsize, xsize, optimizer=tf.train.GradientDescentOptimizer):
        self.learning_rate = tf.Variable(0.0, dtype=tf.float32, trainable=False)
        self.x = tf.placeholder(shape=(batchsize, xsize), dtype=tf.float32, name="x")
        self.y = tf.placeholder(shape=(batchsize,1), dtype=tf.float32, name="y")
        
        #Hidden1
        w = tf.get_variable("w", shape=(xsize, 2))
        b = tf.get_variable("b", shape=(2))
        hidden1 = tf.square(tf.matmul(self.x,w) + b)
        
        #hidden2
        w2 = tf.get_variable("w2", shape=(xsize, 2))
        b2 = tf.get_variable("b2", shape=(2))
        hidden2 = tf.square(tf.matmul(hidden1,w2) + b2)
        
        w3 = tf.get_variable("w3", shape=(2,1))
        b3 = tf.get_variable("b3", shape=(1))
        logit = tf.matmul(hidden2,w3) + b3
        
        self.proba = tf.sigmoid(logit)
        self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=self.y)) 
        self.train = optimizer(self.learning_rate).minimize(self.loss)

In [ ]:
with tf.variable_scope("MLP3", reuse=False, initializer=tf.random_uniform_initializer(minval=-0.05, maxval=0.05)):
    mlp = MLP3(None, 2, optimizer=tf.train.AdamOptimizer)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.assign(mlp.learning_rate, 0.1))

In [ ]:
sess.run(tf.global_variables_initializer())
sess.run(tf.assign(mlp.learning_rate, 1.0))

In [ ]:
for i in range(10000):
    loss, proba,  _  = sess.run([mlp.loss, mlp.proba, mlp.train], feed_dict={mlp.x:x, mlp.y:y}) 
    if i % 1000 == 0:
        accuracy = (np.where(proba >= 0.5, 1, 0) == y).mean()                                                       
        print(loss, accuracy)